In [1]:
!pip install kaggle
!pip install datasets

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -U unsloth unsloth-zoo transformers peft accelerate --quiet
!pip install trl==0.23.0 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.7/358.7 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.4/284.4 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 97.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"fatmaalzhraahmed","key":"540ff34070f54b15a2b81e98896d1372"}'}

In [5]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d fatmaalzhraahmed/shifaa-arabic-medical-q-and-a

Dataset URL: https://www.kaggle.com/datasets/fatmaalzhraahmed/shifaa-arabic-medical-q-and-a
License(s): apache-2.0
  0% 0.00/78.4M [00:00<?, ?B/s]
100% 78.4M/78.4M [00:00<00:00, 827MB/s]


In [7]:
!unzip shifaa-arabic-medical-q-and-a.zip

Archive:  shifaa-arabic-medical-q-and-a.zip
  inflating: Shifaa Arabic Medical Q and A.csv  


In [8]:
import pandas as pd
import numpy as np
import json
from sklearn.model_selection import train_test_split
from datasets import load_dataset,DatasetDict

In [9]:
df=pd.read_csv("/content/Shifaa Arabic Medical Q and A.csv")
df

,Question,Answer
0,أعاني من ألم فوق وبجانب السرة وكأنه طعن بسكين ...,منى إن الأدوية التي تستعملينها (ميتوتركسات وال...
1,عمري 31 سنة ومتزوجة منذ 8 أشهر ولم يحصل الحمل ...,سارة أتفهم مشاعرك ولهفتك على الحمل يا عزيزتي ف...
2,سؤالي عندي طفل يبلغ مع العمر 6 سنوات المشكلة ه...,قد يتسبب الإمساك المزمن عند الطفل بسبب نقص الس...
3,تعرضت لحادث في المباراة وعملت صورة رنين مغناطي...,حبذا لو ذكرت لنا كم مضى على تمزق الرباط وهل هو...
4,أنا شاب عمري 22 عاما أعاني من القولون العصبي م...,لا يمكن لمن يتناول الحلويات والسكريات والمشروب...
...,...,...
84417,أريد أن أعرف ما هي أسباب اسمرار مفاصل اليد حتى...,سميرة المناطق المعرضة للاحتكاك بالمحيط الخارجي...
84418,أعاني من كثافة الشعر على الساعدين والرجلين بشك...,فإن الاختلاف بين الناس وخلقهم وطبائعهم هو أمر ...
84419,ما هو مرض الزلال فقد قمت بفحص البول وأخبرني ال...,الأخ الفاضل أبو زهور الزلال كلمة تطلق على ظهور...
84420,أنا فريال من الجزائر من فضلكم أريد أن أعرف متى...,فريال وبعد عوضك الله بكل خير وجعل صبرك واحتساب...


In [10]:
df.dropna(subset='Question', inplace=True)

In [11]:
data_for_model = []
for index, row in df.iterrows():
    data_for_model.append({
        "instruction": row['Question'],
        "output": row['Answer']
    })

In [12]:
with open('data_ready_for_model.jsonl', 'w', encoding='utf-8') as f:
    for i in data_for_model:
        f.write(json.dumps(i, ensure_ascii=False) + "\n")

In [13]:
full_dataset = load_dataset("json", data_files="/content/data_ready_for_model.jsonl", split='train')

Generating train split: 0 examples [00:00, ? examples/s]

# Split Data Into Train, Test, Val
### With size :


1.   Train : 80% (**67535**)
2.   Validation: 15% (**12663**)
3.   Test :5%  (**4221**)



In [14]:
train_test_split = full_dataset.train_test_split(test_size=0.2, seed=42)

In [15]:
validation_test_split = train_test_split['test'].train_test_split(test_size=0.25, seed=42)

In [16]:
final_dataset = DatasetDict({
    'train': train_test_split['train'],
    'validation': validation_test_split['train'],
    'test': validation_test_split['test']
})

In [17]:
final_dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output'],
        num_rows: 67535
    })
    validation: Dataset({
        features: ['instruction', 'output'],
        num_rows: 12663
    })
    test: Dataset({
        features: ['instruction', 'output'],
        num_rows: 4221
    })
})

In [18]:
train_dataset =  final_dataset['train']
val_dataset = final_dataset['validation']
test_dataset = final_dataset['test']

In [19]:
train_dataset[6]

{'instruction': 'منذ 7 أشهر أجريت عمليه تغيير الصمام الرئوي جراحة في أمريكا والحمد لله بعد العملية لم يكن هناك أي ألم ولكن الآن مع أي مجهود أشعر بألم في قلبي وصدري وصعوبة في التنفس وخفقان هل هو من القلب أم ماذا',
 'output': 'وردة وبعد فلم تذكري نوع العملية التي تم إجراؤها فهل تم تغيير الصمام عن طريق القسطرة أم عن طريق فتح الصدر على كل حال لابد وأنه قد تم شرح مضاعفات العملية لك فأحيانا يحصل قصور في الصمام وأحيانا قد لا يتحسن التضييق في الصمام وأحيانا قد يكون هناك مضاعفات أخرى ولذا ففي مثل حالتك يجب مراجعة الطبيب المتابع لحالتك في مستشفى حمد وبسرعة ليتم معرفة سبب آلام الصدر وضيق النفس والخفقان خاصة إن كانت هذه الأعراض قد تحسنت بعد العملية كما ذكرت في رسالتك الطبيب المعالج العارف لحالتك هو من يستطيع بعد الفحص الطبي وبعد إجراء بعض الصور والإجراءات التشخيصية أن يقدر وضع الصمام وسبب الأعراض الجديدة والله الموفق'}

In [20]:
from huggingface_hub import login
token = "Your Token"
login(token)

In [21]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig,AutoModelForSequenceClassification
from unsloth import FastLanguageModel

/tmp/ipython-input-4100157713.py:2: UserWarning: WARNING: Unsloth should be imported before transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [22]:
bnb_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=False
)

In [23]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B-Instruct",
    max_seq_length=1024,
    token=token,
    quantization_config=bnb_config,
    dtype = None,
    load_in_4bit = True,
)

==((====))==  Unsloth 2025.11.4: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [24]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 64,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 128,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = True,
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.11.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [25]:
arabic_prompt_template = """<|start_header_id|>system<|end_header_id|>
أنت "شِفاء"، مساعد طبي خبير وموثوق. مهمتك هي تحليل الاستفسارات الطبية وتقديم إجابات واضحة، منظمة، ومتعاطفة لمساعدة المستخدمين على فهم حالتهم الصحية بشكل أفضل. استخدم دائمًا تنسيق Markdown (عناوين ونقاط) لتسهيل القراءة.<|eot_id|>

<|start_header_id|>user<|end_header_id|>
{}<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
{}<|eot_id|>
"""


In [26]:
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    outputs      = examples["output"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        text = arabic_prompt_template.format(instruction, output) + tokenizer.eos_token
        texts.append(text)
    return { "text" : texts, }
pass

In [28]:
train_dataset_formatted = train_dataset.map(formatting_prompts_func, batched = True,)
val_dataset_split = val_dataset.train_test_split(test_size=0.6)
val_dataset_50_percent = val_dataset_split['test']
val_dataset_formatted = val_dataset_50_percent.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/67535 [00:00<?, ? examples/s]

Map:   0%|          | 0/7598 [00:00<?, ? examples/s]

In [29]:
train_dataset_formatted['text'][40]

'<|start_header_id|>system<|end_header_id|>\nأنت "شِفاء"، مساعد طبي خبير وموثوق. مهمتك هي تحليل الاستفسارات الطبية وتقديم إجابات واضحة، منظمة، ومتعاطفة لمساعدة المستخدمين على فهم حالتهم الصحية بشكل أفضل. استخدم دائمًا تنسيق Markdown (عناوين ونقاط) لتسهيل القراءة.<|eot_id|>\n\n<|start_header_id|>user<|end_header_id|>\nأنا فتاة عمري 27 سنة أعاني من الذئبة الحمراء وأتعالج باللاكونيل جرعة واحدة وأحيانا أهمله وحللت cbc فظهر تحليلي منخفضا 350 فما سبب الانخفاض علما بأن الذئبة خاملة منذ سنوات والغدة الدرقية جدا خاملة والحديد وفيتامين دال منخفضان كما أتناول الفافرين لعلاج الاكتئاب والوسواس فما سبب الانخفاض في التحليل وكيف أرفعه<|eot_id|>\n\n<|start_header_id|>assistant<|end_header_id|>\nlmmiaa شكرا على تواصلك مع الشبكة الإسلامية إن هذه النسبة من الكريات البيضاء منخفضة بشكل بسيط وقد لا يكون لها أي دلالة فكثيرا ما نجد ذلك عند مرضى الذئبة الحمراء ولا يسبب ذلك أية مشكلة ولا يعني أن هناك زيادة في حصول التهابات خاصة إنك لا تتناولين أي أدوية يمكن أن تسبب نقص الكريات البيضاء ولذلك نطمئن المريضة ونعيد ا

In [30]:
print(train_dataset_formatted['text'][40])

<|start_header_id|>system<|end_header_id|>
أنت "شِفاء"، مساعد طبي خبير وموثوق. مهمتك هي تحليل الاستفسارات الطبية وتقديم إجابات واضحة، منظمة، ومتعاطفة لمساعدة المستخدمين على فهم حالتهم الصحية بشكل أفضل. استخدم دائمًا تنسيق Markdown (عناوين ونقاط) لتسهيل القراءة.<|eot_id|>

<|start_header_id|>user<|end_header_id|>
أنا فتاة عمري 27 سنة أعاني من الذئبة الحمراء وأتعالج باللاكونيل جرعة واحدة وأحيانا أهمله وحللت cbc فظهر تحليلي منخفضا 350 فما سبب الانخفاض علما بأن الذئبة خاملة منذ سنوات والغدة الدرقية جدا خاملة والحديد وفيتامين دال منخفضان كما أتناول الفافرين لعلاج الاكتئاب والوسواس فما سبب الانخفاض في التحليل وكيف أرفعه<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
lmmiaa شكرا على تواصلك مع الشبكة الإسلامية إن هذه النسبة من الكريات البيضاء منخفضة بشكل بسيط وقد لا يكون لها أي دلالة فكثيرا ما نجد ذلك عند مرضى الذئبة الحمراء ولا يسبب ذلك أية مشكلة ولا يعني أن هناك زيادة في حصول التهابات خاصة إنك لا تتناولين أي أدوية يمكن أن تسبب نقص الكريات البيضاء ولذلك نطمئن المريضة ونعيد التحليل ب

In [31]:
from trl import SFTTrainer
from transformers import TrainingArguments
import torch

In [32]:
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=1024,
        padding=False,
    )

train_tokenized = train_dataset_formatted.map(
    tokenize_function,
    batched=True,
    remove_columns=train_dataset_formatted.column_names,
)

val_tokenized = val_dataset_formatted.map(
    tokenize_function,
    batched=True,
    remove_columns=val_dataset_formatted.column_names,
)


Map:   0%|          | 0/67535 [00:00<?, ? examples/s]

Map:   0%|          | 0/7598 [00:00<?, ? examples/s]

In [33]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

In [34]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/llama_arabic_medical_q_and_a_model",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=2,
    warmup_steps=10,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),
    logging_steps=10,
    optim = "adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    eval_strategy="steps",
    eval_steps=1000,
    save_strategy="steps",
    save_steps=50,
    report_to="wandb",
    run_name=f"llama3.1-8b-lr-{2e-4}-ep-{1}",
    remove_unused_columns=False,
)

In [36]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    data_collator=data_collator,
)
trainer.train(resume_from_checkpoint="/content/drive/MyDrive/llama_arabic_medical_q_and_a_model/checkpoint-2111")

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 67,535 | Num Epochs = 1 | Total steps = 2,111
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 4 x 1) = 32
 "-____-"     Trainable parameters = 167,772,160 of 8,198,033,408 (2.05% trained)
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: fatmaalzhraahmed837 (fatmaalzhraahmed837-helwan-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Step,Training Loss,Validation Loss


TrainOutput(global_step=2111, training_loss=0.0, metrics={'train_runtime': 128.3149, 'train_samples_per_second': 526.322, 'train_steps_per_second': 16.452, 'total_flos': 3.131012143157551e+18, 'train_loss': 0.0, 'epoch': 1.0})

In [37]:
trainer.evaluate()

Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


{'eval_loss': 2.8331665992736816,
 'eval_runtime': 7788.2244,
 'eval_samples_per_second': 0.976,
 'eval_steps_per_second': 0.488,
 'epoch': 1.0}